# TSA Chapter 7: Case Study - Unit Root Tests

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_case_unit_root/TSA_ch7_case_unit_root.ipynb)

ACF analysis of levels (slow decay), first differences (rapid decay), and spread (stationary) using FRED GS10 and TB3MS data.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'
gs10 = web.DataReader('GS10', 'fred', start, end).dropna()
tb3ms = web.DataReader('TB3MS', 'fred', start, end).dropna()
data = pd.DataFrame({'GS10': gs10['GS10'], 'TB3MS': tb3ms['TB3MS']}).dropna()
spread = data['GS10'] - data['TB3MS']
print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# Compute ACFs
nlags = 30
acf_level = acf(data['GS10'], nlags=nlags, fft=True)
acf_diff = acf(data['GS10'].diff().dropna(), nlags=nlags, fft=True)
acf_spread = acf(spread, nlags=nlags, fft=True)

# ADF tests
adf_level = adfuller(data['GS10'])
adf_diff = adfuller(data['GS10'].diff().dropna())
adf_spread = adfuller(spread)

# Confidence band
ci = 1.96 / np.sqrt(len(data))

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Panel 1: ACF of levels (slow decay)
axes[0].bar(range(nlags + 1), acf_level, color=BLUE, width=0.6)
axes[0].axhline(ci, color=RED, ls='--', lw=0.8)
axes[0].axhline(-ci, color=RED, ls='--', lw=0.8)
axes[0].axhline(0, color='black', lw=0.5)
axes[0].set_title(f'ACF: GS10 Level (ADF p={adf_level[1]:.3f})', fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')

# Panel 2: ACF of first differences (rapid decay)
axes[1].bar(range(nlags + 1), acf_diff, color=RED, width=0.6)
axes[1].axhline(ci, color=GRAY, ls='--', lw=0.8)
axes[1].axhline(-ci, color=GRAY, ls='--', lw=0.8)
axes[1].axhline(0, color='black', lw=0.5)
axes[1].set_title(f'ACF: $\\Delta$GS10 (ADF p={adf_diff[1]:.4f})', fontweight='bold')
axes[1].set_xlabel('Lag')

# Panel 3: ACF of spread (stationary)
axes[2].bar(range(nlags + 1), acf_spread, color=GREEN, width=0.6)
axes[2].axhline(ci, color=GRAY, ls='--', lw=0.8)
axes[2].axhline(-ci, color=GRAY, ls='--', lw=0.8)
axes[2].axhline(0, color='black', lw=0.5)
axes[2].set_title(f'ACF: Spread (ADF p={adf_spread[1]:.4f})', fontweight='bold')
axes[2].set_xlabel('Lag')

plt.tight_layout()
save_chart(fig, 'ch7_case_unit_root')
plt.show()

print(f'\nADF Results:')
print(f'  GS10 level:  t-stat={adf_level[0]:.3f}, p={adf_level[1]:.4f} -> {"Stationary" if adf_level[1] < 0.05 else "Non-stationary"}')
print(f'  GS10 diff:   t-stat={adf_diff[0]:.3f}, p={adf_diff[1]:.4f} -> {"Stationary" if adf_diff[1] < 0.05 else "Non-stationary"}')
print(f'  Spread:      t-stat={adf_spread[0]:.3f}, p={adf_spread[1]:.4f} -> {"Stationary" if adf_spread[1] < 0.05 else "Non-stationary"}')